In [ ]:
import pandas as pd
import numpy as np 
import pylab 
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
from numpy.random import seed
from numpy.random import randn
from scipy.stats import shapiro
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from IPython.display import Image
%matplotlib inline

# Import Data

In [ ]:
df = pd.read_csv('wdbc.csv', names=['ID','Diagnosis','MeanRadius','MeanTexture','MeanPerimeter','MeanArea',
                                    'MeanSmoothness','MeanCompactness','MeanConcavity','MeanConcavePoints',
                                    'MeanSymmetry','MeanFractalDimension','RadiusSE','TextureSE','PerimeterSE',
                                    'AreaSE','SmoothnessSE','CompactnessSE','ConcavitySE','ConcavePointsSE',
                                    'SymmetrySE','FractalDimensionSE','WorstRadius','WorstTexture','WorstPerimeter',
                                    'WorstArea','WorstSmoothness','WorstCompactness','WorstConcavity',
                                    'WorstConcave Points','WorstSymmetry','WorstFractalDimension'])

# EDA

In [ ]:
type(df)
df.info() #no null values

In [ ]:
df.head(10)

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
df['Diagnosis'].value_counts().plot(kind='bar');

## What are the mean, median and standard deviation of the “perimeter” feature?

In [ ]:
# Mean of the MeanPerimeter column is 91.97, from df.describe() table above.
# Median of MeanPerimeter is 86.24, and standard deviation of MeanPerimeter is 24.30.
df.MeanPerimeter.plot(kind='box');

## Is the first feature in this data set (the “radius”) normally distributed? Please quantitatively define you answer. If not, what might be a more appropriate distribution?

In [ ]:
df.MeanRadius.plot(kind='box');

In [ ]:
df.MeanRadius.plot(kind='hist', bins=20);
# Based on a histogram of the mean radius values, this feature does not look normally distributed.
# This distribution is right skewed.

In [ ]:
stats.probplot(df.MeanRadius, dist="norm", plot=pylab)
pylab.show()
# The QQ plot suggests the radius feature is not normal, as the blue data starts above the normal line, goes below, and 
# then goes back above, indicating some underlying patterns that are not well represented by a normal distribution.

In [ ]:
# Shapiro-Wilk Test - a hypothesis test for if the distibution is normal
print('Null hypothesis: the radius is normally distributed.')
seed(1234)
stat, p = shapiro(df.MeanRadius)
print('Test Statistic = %.3f, p-value = %.6f.' % (stat, p))
# The p-value is neglibile for this test of the null hypothesis, and therefore at any level of signifcance we can reject the
# null hypothesis, and conclude that the radius is not normally distributed.

### A more appropriate distribution might be a Gamma distribution, or possibly an F distribution.

In [ ]:
x = np.linspace(0.001, 35, 5000)
f, ax_arr = plt.subplots(2, 2, figsize=(14,7))

# Histogram
ax_arr[0,0].hist(df.MeanRadius, bins=20)
ax_arr[0,0].set_title('Histogram of MeanRadius')

# F distribution
yF=stats.f.pdf(x, 10, 100, 0, 11)
ax_arr[0,1].plot(x, yF);
ax_arr[0,1].set_title('F distribution')
# I played around with the parameters of the F distribution pdf, and found this combination gives a distribution which looks
# similar to our histogram of the MeanRadius.

# Gamma distribution
yGamma=stats.gamma.pdf(x, 14)
ax_arr[1,0].plot(x, yGamma);
ax_arr[1,0].set_title('Gamma distribution')
# The mean of MeanRadius is 14.13. When I use 14 as the shape parameter here, this gamma distribution looks similar to our 
# histogram of MeanRadius.

ax_arr[1,1].axis('off')
plt.show()

In [ ]:
# One could check if these Gamma/F distributions are appropriate using a Kolmogorov–Smirnov test.

## Train a classifier to predict the diagnosis of malignant or benign. Please compare the results of two classifiers e.g. SVM, logistic regression, decision tree etc.

In [ ]:
# Deal with missing values (none), outliers and feature engineering.
# Look at ranges of all Mean features - do I need to standardize?
dfMeans = df[['Diagnosis','MeanRadius','MeanTexture','MeanPerimeter','MeanArea',
                                    'MeanSmoothness','MeanCompactness','MeanConcavity','MeanConcavePoints',
                                    'MeanSymmetry','MeanFractalDimension']]

In [ ]:
dfMeans.describe()
# Many values always <1 but others larger, in particular MeanArea has a mean of 654.89 with largest value 2501.

In [ ]:
# Expect correlation/collinearity between Radius, Area, Perimeter and Compactness (perimeter^2 / area - 1.0)/
# As area takes such large values, could log or sqrt transform, but as area = pi * radius**2, could just use radius instead.
f, ax_arr = plt.subplots(3, 2, figsize=(14,7))

ax_arr[0,0].scatter(dfMeans.MeanRadius, dfMeans.MeanArea);
ax_arr[0,0].set_xlabel('MeanRadius')
ax_arr[0,0].set_ylabel('MeanArea')

ax_arr[0,1].scatter(dfMeans.MeanRadius, dfMeans.MeanPerimeter); 
ax_arr[0,1].set_xlabel('MeanRadius')
ax_arr[0,1].set_ylabel('MeanPerimeter')

ax_arr[1,0].scatter(dfMeans.MeanRadius, dfMeans.MeanCompactness);
ax_arr[1,0].set_xlabel('MeanRadius')
ax_arr[1,0].set_ylabel('MeanCompactness')

ax_arr[1,1].scatter(dfMeans.MeanArea, dfMeans.MeanPerimeter);
ax_arr[1,1].set_xlabel('MeanArea')
ax_arr[1,1].set_ylabel('MeanPerimeter')

ax_arr[2,0].scatter(dfMeans.MeanArea, dfMeans.MeanCompactness);
ax_arr[2,0].set_xlabel('MeanArea')
ax_arr[2,0].set_ylabel('MeanCompactness')

ax_arr[2,1].scatter(dfMeans.MeanPerimeter, dfMeans.MeanCompactness);
ax_arr[2,1].set_xlabel('MeanPerimeter')
ax_arr[2,1].set_ylabel('MeanCompactness')

plt.tight_layout()
plt.show()

# Suggests collinearity between radius and area, radius and perimeter, and perimeter and area.
# In context, I think radius is best feature to use out of all of these. Then will use compactness as well.

In [ ]:
def outlierPlots(column, columnName):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,4))
    ax1.hist(column, bins=20);
    ax1.set_title(columnName)
    ax2.boxplot(column);
    ax2.set_title(columnName)
    plt.show()

outlierPlots(dfMeans.MeanRadius, 'MeanRadius')
outlierPlots(dfMeans.MeanTexture, 'MeanTexture')
outlierPlots(dfMeans.MeanSmoothness, 'MeanSmoothness')
outlierPlots(dfMeans.MeanCompactness, 'MeanCompactness')
outlierPlots(dfMeans.MeanConcavity, 'MeanConcavity')
outlierPlots(dfMeans.MeanConcavePoints, 'MeanConcavePoints')
outlierPlots(dfMeans.MeanSymmetry, 'MeanSymmetry')
outlierPlots(dfMeans.MeanFractalDimension, 'MeanFractalDimension')

In [ ]:
outlierRange = (dfMeans.quantile(0.75) - dfMeans.quantile(0.25)) * 1.5
outlierHighBoundary = dfMeans.quantile(0.75) + outlierRange
outlierLowBoundary = dfMeans.quantile(0.25) - outlierRange

In [ ]:
outliers = dfMeans[(dfMeans.MeanFractalDimension > outlierHighBoundary.MeanFractalDimension) & (dfMeans.MeanFractalDimension < outlierLowBoundary.MeanFractalDimension)]
# no outliers (under this definition) for any of the 'Mean...' variables
len(outliers)

### Split data into test and training datasets: 80/20

In [ ]:
# Split data into test and training datasets: 80/20
X = dfMeans.loc[:,'MeanRadius':].as_matrix()
y = dfMeans['Diagnosis'].ravel()

In [ ]:
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
print('Diagnosis counts in train: "M" has {0:.0f}, "B" has {1:.0f}. Ratio of "M" to "B" is {2:.3f}.'
      .format(np.count_nonzero(y_train == 'M'),np.count_nonzero(y_train == 'B'),
             np.count_nonzero(y_train == 'M')/np.count_nonzero(y_train == 'B')))
print('Diagnosis counts in train: "M" has {0:.0f}, "B" has {1:.0f}. Ratio of "M" to "B" is {2:.3f}.'
      .format(np.count_nonzero(y_test == 'M'),np.count_nonzero(y_test == 'B'),
             np.count_nonzero(y_test == 'M')/np.count_nonzero(y_test == 'B')))

## Logistic Regression Model with all Mean variables

In [ ]:
model_lr = LogisticRegression(random_state=1)

In [ ]:
model_lr.fit(X_train, y_train)

In [ ]:
print('Score for logisitic regression model is {0:.2f}'.format(model_lr.score(X_test, y_test)))

In [ ]:
#Performance metrics
print('Accuracy for logisitic regression model is {0:.2f}'.format(accuracy_score(y_test, model_lr.predict(X_test))))
print('Confusion matrix for logisitic regression model is \n {0}'.format(confusion_matrix(y_test, model_lr.predict(X_test))))
# Need to change 'M', 'B' to numeric values for precision and recall scores.

In [ ]:
# Model coefficients for each feature
model_lr.coef_

## Logistic Regression Model with fewer Mean variables

In [ ]:
# Removing collinear varibles - keeping radius only
dfMeansFewer = dfMeans[dfMeans.columns.difference(['MeanArea', 'MeanPerimeter'])]
dfMeansFewer.shape

In [ ]:
X2 = dfMeansFewer.loc[:,'MeanCompactness':].as_matrix()
y2 = dfMeansFewer['Diagnosis'].ravel()
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=1)

In [ ]:
model_lr2 = LogisticRegression(random_state=1)
model_lr2.fit(X_train2, y_train2)

In [ ]:
print('Score for logisitic regression model is {0:.2f}'.format(model_lr2.score(X_test2, y_test2)))
print('Accuracy for logisitic regression model is {0:.2f}'.format(accuracy_score(y_test2, model_lr2.predict(X_test2))))
print('Confusion matrix for logisitic regression model is \n {0}'.format(confusion_matrix(y_test2, model_lr2.predict(X_test2))))
# Using fewer variables gives same accuracy scores.

### Cross Validation of Logistic Regression Model with fewer Mean variables

In [ ]:
model_lr3 = LogisticRegression(random_state=1)
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']}
# 3 fold cross validation
clf = GridSearchCV(model_lr3, param_grid=parameters, cv=3)

In [ ]:
clf.fit(X_train2, y_train2)

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

In [ ]:
print('Score for logistic regression with hyperparameter optimization is {0:.02f}'.format(clf.score(X_test2, y_test2)))
print('Accuracy for logistic regression with hyperparameter optimization is {0:.02f}'
      .format(accuracy_score(y_test2, clf.predict(X_test2))))
print('Confusion matrix for logistic regression with hyperparameter optimization is \n {0}'
      .format(confusion_matrix(y_test2, clf.predict(X_test2))))

### Cross Validation of Logistic Regression Model with fewer Mean variables (standardized)

In [ ]:
# Standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train2)
X_test_scaled = scaler.transform(X_test2)

In [ ]:
model_lr4 = LogisticRegression(random_state=1)
clf2 = GridSearchCV(model_lr4, param_grid=parameters, cv=3)
clf2.fit(X_train_scaled, y_train2)

In [ ]:
clf2.best_score_

In [ ]:
print('Score for logistic regression with hyperparameter optimization and standardized variables is {0:.02f}'
      .format(clf2.score(X_test_scaled, y_test2)))
print('Accuracy for logistic regression with hyperparameter optimization and standardized variables is {0:.02f}'
      .format(accuracy_score(y_test2, clf2.predict(X_test_scaled))))
print('Confusion matrix for logistic regression with hyperparameter optimization and standardized variables is \n {0}'
      .format(confusion_matrix(y_test2, clf2.predict(X_test_scaled))))

In [ ]:
# So standardizing variables gives a slightly worse 'best' score, but better 'score' and accuracy.

## Decision Tree

In [ ]:
dtree = DecisionTreeClassifier(max_depth=10, random_state=1, max_features=None, min_samples_leaf=15)
dfit = dtree.fit(X_train2, y_train2)
dfit

In [ ]:
y_pred = dtree.predict(X_test2)

In [ ]:
print('Score for decision tree model is {0:.2f}'.format(model_lr2.score(X_test2, y_test2)))
print('Accuracy for decision tree model is {0:.2f}'.format(accuracy_score(y_test2, y_pred)))
print('Confusion matrix for decision tree model is \n {0}'.format(confusion_matrix(y_test2, y_pred)))

#### Decision Tree Visualization

In [ ]:
export_graphviz(dfit, out_file='tree.dot', rounded=True, proportion=False, filled=True)
!dot -Tpng tree.dot -o tree.png -Gdpi=600
Image(filename = 'tree.png')

### Cross Validation of Decision Tree

In [ ]:
treeParameters = {'max_depth':[8, 10, 12], 'min_samples_leaf': [10, 15, 20]}
clf3 = GridSearchCV(dtree, param_grid=treeParameters, cv=3)
clf3.fit(X_train2, y_train2)

In [ ]:
clf3.best_score_

In [ ]:
print('Score for cross validation of decision tree is {0:.02f}'.format(clf3.score(X_test2, y_test2)))
print('Accuracy for decision tree model is {0:.2f}'.format(accuracy_score(y_test2, dtree.predict(X_test2))))
print('Confusion matrix for decision tree model is \n {0}'.format(confusion_matrix(y_test2, dtree.predict(X_test2))))

## Conclusion:

#### In this example, the model that performed best was found when doing Cross Validation of Logistic Regression Model with fewer Mean variables (standardized).
#### Also, it is not a surprise that using cross validation to compare different model parameters improved performance metrics in all scenarios.